### EECS 730 Project - 4

#### Steps followed
1. 
2. 
3. 
4. 
5. 
6. 

#### Instructions to run
1. 
2. 
3. 
4. 
5. 

#### Import relevant packages

In [1]:
# import packages
import os
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Align
import numpy as np
import pandas as pd
import math
import sys
#import dask.dataframe as dd
#from dask.multiprocessing import get
#from pydna.common_sub_strings import terminal_overlap
#from pydna.assembly import Assembly
#from pydna.dseqrecord import Dseqrecord

# Print versions
np.set_printoptions(threshold=sys.maxsize)
print('The Biopython version is {}..'.format(Bio.__version__))

The Biopython version is 1.78..


#### Create the paths for reference files

In [2]:
# Take file names as input. # Change the file name accordingly.
ffasta = 'AAA.fa' 
fhmm = 'AAA.hmm'

# Set the local paths for data
path = r'C:\Users\pmspr\Documents\HS\MS\Sem 4\EECS 730\Bioinformatics\Project 4\Docs'
fasta = os.path.join(path, ffasta) 
hmm   = os.path.join(path, fhmm)
test  = os.path.join(path,'text.csv')

In [3]:
#Read hmm file in to dataframes
states = [];tran_cols = [];
match = []; insert = []; trans = [];
hmmoffset = 0; holdstate = ' ';
state_cnt = 0;lineoffset = 0;

with open(hmm) as flhmm:
    for line in flhmm:
        if (line.strip()[0:3] == '//'):
            break
            
        if (line[0:4].strip() == 'HMM'):
            states = [s for s in line.split(' ') if (s not in ['','HMM','\n'])]
            state_cnt = len(states)
            hmmoffset += 1
            continue 
        if (hmmoffset == 1):
            tran_cols = [s for s in line.strip('\n').split(' ') if (s not in ['','\n'])]
            hmmoffset += 1
            continue
        if (hmmoffset == 2):
            lineoffset += 1
            if (lineoffset == 1):
                match_row = []
                match_row = [s for s in line.strip('\n').split(' ') if (s not in ['','\n'])][0:state_cnt+1]
                holdstate = match_row[0]
                match.append(match_row)
            if (lineoffset == 2):
                insert.append([holdstate]+[s for s in line.strip('\n').split(' ') if (s not in ['','\n'])][0:state_cnt+1])
            if (lineoffset == 3):
                trans.append([holdstate]+[s for s in line.strip('\n').split(' ') if (s not in ['','\n'])][0:len(tran_cols)+1])
                lineoffset = 0; holdstate = ' ';
            
match_df = pd.DataFrame(match, columns=['state']+states); match_df['state'].iloc[0] = 0;
insert_df = pd.DataFrame(insert, columns=['state']+states); insert_df['state'].iloc[0] = 0;
trans_df =  pd.DataFrame(trans,columns=['state']+tran_cols); trans_df['state'].iloc[0] = 0;
trans_df.loc[trans_df['state'] == 0] = 1;

# Display sample dataframes
display(match_df.head())
display(insert_df.tail())
display(trans_df.head())

,state,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
0,0,2.52604,4.42416,2.95323,2.65774,3.30769,2.77257,3.88316,2.70171,2.68015,...,3.63090,3.12983,3.35132,3.16734,2.88091,2.66110,2.77599,2.64066,5.09582,3.81495
1,1,2.67876,4.02341,5.04584,4.43081,3.28407,4.25179,4.58598,1.96692,4.21589,...,2.24040,4.41437,3.96304,4.34710,4.22410,3.41343,3.21354,1.53323,5.07918,3.39642
2,2,3.08766,3.31586,5.08732,4.47351,3.39603,3.97708,4.63574,2.19488,4.26134,...,3.15016,4.46297,4.65996,4.38884,4.27052,3.61471,3.35994,2.50009,5.12115,2.76043
3,3,3.45894,4.80484,5.41061,4.80074,2.11887,4.65865,3.72880,2.36028,4.59943,...,3.25424,4.81739,4.96559,4.67891,4.59615,3.97997,3.68534,2.05256,5.37563,4.23707
4,4,2.64322,3.99168,4.60427,3.48960,2.77546,4.03531,2.66459,2.49429,3.65970,...,3.64256,3.87334,4.55149,3.92136,4.03884,2.44837,2.80149,2.27618,3.94482,1.75167


,state,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
128,128,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
129,129,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
130,130,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
131,131,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
132,132,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503


,state,m->m,m->i,m->d,i->m,i->i,d->m,d->d
0,1,1,1,1,1,1,1,1
1,1,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510
2,2,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510
3,3,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510
4,4,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510


In [4]:
# Read the sequence file
seql = []
with open(fasta) as flfst:
    for line in flfst:
        if (line.strip()[0] != '>'):
            seq = line.strip('\n')
seql = [seq[s] for s in range(len(seq))]
print('Input sequence with length {}:\n{}'.format(len(seq),seq))

Input sequence with length 130:
VLFEGPPGTGKTSCARVIANQAGIPLLYVPLEAVMSKYYGESERLLGAVFSQANELPDGAIIFLDEIDAFAISRDSEMHEATRRVLSVLLRQIDGFEQEKKVVVIAATNRKQDLDPALISRFDSMIMFDL


In [30]:
# Calculate matrices
ncols = match_df.shape[0] - 1     #Ignore compo
nrows = len(seq) 

# Declare and initialize

# Match matrix
vm = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)
vm[0,0] = 0; vm[0,1:] = -1000; vm[1:,0] = -1000
vmt = np.array([['x' for i in range(0,ncols+1)]for j in range(0,nrows+1)], dtype=np.object)

# Insert matrix
vi = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)
vit = np.array([['x' for i in range(0,ncols+1)]for j in range(0,nrows+1)], dtype=np.object)
vi[0:,0] = -1000

# Delete matrix
vd = np.array([0,0], dtype=np.float64)
vd = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)
vdt = np.array([['x' for i in range(0,ncols+1)]for j in range(0,nrows+1)], dtype=np.object)
vd[0,0:] = -1000

# Terminate matrix
vt = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)

# Trace matrix
tb = np.array([['x' for i in range(0,ncols+1)]for j in range(0,nrows+1)], dtype=np.object)

# Fill the matrice
n = 0.0; n1 = 0.0; n2 = 0.0; n3 = 0.0;
for j in range(1, nrows+1):
    for i in range(1,ncols+1):
        
        # Match matrix
        if (trans_df['m->m'].iloc[j-1] == '*'):
            trans_df['m->m'].iloc[j-1] = -1000
        
        if (trans_df['i->m'].iloc[j-1] == '*'):
            trans_df['i->m'].iloc[j-1] = -1000
        
        if (trans_df['d->m'].iloc[j-1] == '*'):
            trans_df['d->m'].iloc[j-1] = -1000
            
        n1 = vm[j-1,i-1] + float(trans_df['m->m'].iloc[j-1])
        n2 = vi[j-1,i-1] + float(trans_df['i->m'].iloc[j-1])
        n3 = vd[j-1,i-1] + float(trans_df['d->m'].iloc[j-1])
        n  = float(match_df[seq[j-1]].iloc[i]) + np.log(seql.count(seq[j-1]))
        maxar = np.array([n1,n2,n3],dtype=np.float64)
        vm[j,i] = n + np.amax(maxar)
        
        # Match trace matrix
        if(np.argmax(maxar) == 0):
            vmt[j,i] = 'd'
        if(np.argmax(maxar) == 1):
            vmt[j,i] = 'u'
        if(np.argmax(maxar) == 2):
            vmt[j,i] = 'l'
        
        # Insert matrix
        if (trans_df['m->i'].iloc[j] == '*'):
            trans_df['m->i'].iloc[j] = -1000
        
        if (trans_df['i->i'].iloc[j] == '*'):
            trans_df['i->i'].iloc[j] = -1000
            
        n1 = vm[j,i-1] + float(trans_df['m->i'].iloc[j])
        n2 = vi[j,i-1] + float(trans_df['i->i'].iloc[j])
#         n3 = vd[j,i-1]
        n  = float(insert_df[seq[j-1]].iloc[i]) + np.log(seql.count(seq[j-1]))
        maxar = np.array([n1,n2],dtype=np.float64)
        vi[j,i] = n + np.amax(maxar)
        
        # Insert trace matrix
        if(np.argmax(maxar) == 0):
            vit[j,i] = 'd'
        if(np.argmax(maxar) == 1):
            vit[j,i] = 'u'
#         if(np.argmax(maxar) == 2):
#             vit[j,i] = 'l'
        
        # Delete matrix
        if (trans_df['m->d'].iloc[j-1] == '*'):
            trans_df['m->d'].iloc[j-1] = -1000
        
        if (trans_df['d->d'].iloc[j-1] == '*'):
            trans_df['d->d'].iloc[j-1] = -1000
            
        n1 = vm[j-1,i] + float(trans_df['m->d'].iloc[j-1])
#         n2 = vi[j-1,i] 
        n3 = vd[j-1,i] + float(trans_df['d->d'].iloc[j-1])
        maxar = np.array([n1,n3],dtype=np.float64)
        vd[j,i] = np.amax(maxar)
        
        # Delete trace matrix
        if(np.argmax(maxar) == 0):
            vdt[j,i] = 'd'
#         if(np.argmax(maxar) == 1):
#             vdt[j,i] = 'u'
        if(np.argmax(maxar) == 1):
            vdt[j,i] = 'l'
        
        # Termination matrix
        maxar = np.array([vm[j,i],vi[j,i],vd[j,i]],dtype=np.float64)
        vt[j,i] = np.amax(maxar)
        
        # Traceback matrix
        if(np.argmax(maxar) == 0):
            tb[j,i] = 'M'
        if(np.argmax(maxar) == 1):
            tb[j,i] = 'I'
        if(np.argmax(maxar) == 2):
            tb[j,i] = 'D'
            
test_df = pd.DataFrame(vdt)
t=test_df.to_csv(test,index=False)

In [40]:
# Trace back
i = tb.shape[0]-1; j = tb.shape[1]-1
st = match_df.columns[1:]

a0 = []; a1 = []; a2 = [];
while(tb[i,j] != 'x'):
    #a0.append(seq[i-1])
    if(tb[i,j] == 'M'):
        a0.append(seq[i-1])
        if(vmt[i,j] == 'd'):
            a1.append('M')
        if(vmt[i,j] == 'u'):
            a1.append('I')
        if(vmt[i,j] == 'l'):
            a1.append('D')
        i=i-1; j=j-1
    if(tb[i,j] == 'I'):
        if(vit[i,j] == 'd'):
            a1.append('M')
        if(vit[i,j] == 'u'):
            a1.append('I')
        i=i-1;
    if(tb[i,j] == 'D'):
        if(vdt[i,j] == 'd'):
            a1.append('M')
        if(vdt[i,j] == 'l'):
            a1.append('D')
        j=j-1;

# Reverse the arrays
as0 = ''.join(a0[::-1]); as1 = ''.join(a1[::-1])
print(as0)
print(as1)
print(as0)

LS
DDDDDDDDDDDDDDDDMDDDDDDDDMMDMDMDDMDDDMMMDDMMMMDMDMMDMMMMMMMMMMMMMMIMMMMMMIMMMDMMMMMMMMMMMMMMMMMMDMMMMMMMMMMMMMDMIMMMMMMMDMMDDMDDDMMDDDMDDMIDMMIMMIMDD
LS
149


In [41]:
# Trace back
i = tb.shape[0]-1; j = tb.shape[1]-1
st = match_df.columns[1:]

a0 = []; a1 = []; a2 = [];
while(tb[i,j] != 'x'):
    a0.append(seq[i-1])
    maxar = vt[i,:]
    j = np.argmax(maxar)
    a1.append(tb[i,j])
    i = i - 1
    
# Reverse the arrays
as0 = ''.join(a0[::-1]); as1 = ''.join(a1[::-1])
print(as0)
print(as1)
print(as0)

IndexError: index 265 is out of bounds for axis 1 with size 133